In [0]:
# PART 1: INGESTION
import os
from google.cloud import storage

# CONFIGURATION
DBFS_KEY_PATH = "dbfs:/FileStore/tables/gcp_key.json" 

BUCKET_NAME = "paysim-datalake"
FILE_NAME = "paysim.csv"

# Path ชั่วคราวบน Driver (Local)
LOCAL_KEY_PATH = "/tmp/gcp_key.json"
LOCAL_DATA_PATH = f"/tmp/{FILE_NAME}"
# Path บน DBFS (เพื่อให้ Spark อ่านได้)
DBFS_DATA_PATH = f"dbfs:/FileStore/raw/{FILE_NAME}"

print("Step 1: Secure Ingestion")

try:
    # 1. Setup Credentials (Copy Key from DBFS -> Local)
    dbutils.fs.cp(DBFS_KEY_PATH, f"file:{LOCAL_KEY_PATH}")
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = LOCAL_KEY_PATH
    print("Credentials Loaded.")

    # 2. Download Data from GCS to Local Driver
    print(f"Downloading {FILE_NAME}")
    storage_client = storage.Client()
    bucket = storage_client.bucket(BUCKET_NAME)
    blob = bucket.blob(FILE_NAME)
    blob.download_to_filename(LOCAL_DATA_PATH)
    
    # 3. Move Data to DBFS (เพื่อให้ใช้ spark.read ได้)
    dbutils.fs.mv(f"file:{LOCAL_DATA_PATH}", DBFS_DATA_PATH)
    
    # 4. Read with Spark
    print("Reading CSV with Spark")
    df = spark.read.format("csv") \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .load(DBFS_DATA_PATH)
    
    print("Ingestion successful.")

    # Preview Data
    print("Previewing Data:")
    display(df.limit(10))

In [0]:
# PART 2: BRONZE LAYER
    print("Step 1: Creating Bronze Layer")
    
    # 1. Check Schema
    print("Schema:")
    df.printSchema()
    
    # 2. Count Rows
    print(f"Total Rows: {df.count():,}")
    
    # 3. Save as Delta Table
    print("Creating Bronze Table: 'paysim_bronze'")
    spark.sql("DROP TABLE IF EXISTS paysim_bronze")
    df.write.format("delta").mode("overwrite").saveAsTable("paysim_bronze")
    
    print("Bronze Layer Created")
    
    # Cleanup Local Key
    if os.path.exists(LOCAL_KEY_PATH): os.remove(LOCAL_KEY_PATH)

except Exception as e:
    print(f"Error: {e}")
    if os.path.exists(LOCAL_KEY_PATH): os.remove(LOCAL_KEY_PATH)

In [0]:
# PART 3: SILVER LAYER (CLEANSED & QUALITY CHECK)
from pyspark.sql.functions import col, lit

print("Step 2: Creating Silver Layer")

# 1. Read from Bronze Delta
df_bronze = spark.table("paysim_bronze")

# Step A: Data Quality Check
# Rule: Amount must be non-negative. Negative values indicate system errors.
dq_condition = col("amount") >= 0

# Separate Bad Data (Quarantine) from "Valid Data"
df_valid_amount = df_bronze.filter(dq_condition)
df_quarantine = df_bronze.filter(~dq_condition).withColumn("dq_issue", lit("Negative Amount"))

# Step B: Business Scope Separation
# Rule: We strictly focus on TRANSFER and CASH_OUT for fraud detection.
# Other types (PAYMENT, DEBIT) are valid but out of scope for this model.
scope_condition = col("type").isin("TRANSFER", "CASH_OUT")

# 1. Target Data (Silver): Ready for ML processing
df_silver = df_valid_amount.filter(scope_condition)

# 2. Out-of-Scope Data (Others): Archived for future analytics
df_others = df_valid_amount.filter(~scope_condition)

# 4. Type Casting for Performance (Applied to Silver Table)
# Casting String to Double/Integer for efficient calculation
df_silver = df_silver.withColumn("amount", col("amount").cast("double")) \
                   .withColumn("oldbalanceOrg", col("oldbalanceOrg").cast("double")) \
                   .withColumn("newbalanceOrig", col("newbalanceOrig").cast("double")) \
                   .withColumn("oldbalanceDest", col("oldbalanceDest").cast("double")) \
                   .withColumn("newbalanceDest", col("newbalanceDest").cast("double")) \
                   .withColumn("isFraud", col("isFraud").cast("integer"))

# 5. Save as Delta Tables
print("Creating Silver Table (Target): 'paysim_silver'")
spark.sql("DROP TABLE IF EXISTS paysim_silver")
df_silver.write.format("delta").mode("overwrite").saveAsTable("paysim_silver")

print("Creating Others Table (Out of Scope): 'paysim_others'")
spark.sql("DROP TABLE IF EXISTS paysim_others")
df_others.write.format("delta").mode("overwrite").saveAsTable("paysim_others")

print("Creating Quarantine Table (Bad Data): 'paysim_quarantine'")
spark.sql("DROP TABLE IF EXISTS paysim_quarantine")
df_quarantine.write.format("delta").mode("overwrite").saveAsTable("paysim_quarantine")

# Print Statistics
print(f"""
Stats Summary:
- Silver (ML Ready):   {spark.table('paysim_silver').count():,} rows
- Others (Ignored):    {spark.table('paysim_others').count():,} rows
- Quarantine (Bad):    {spark.table('paysim_quarantine').count():,} rows
""")

In [0]:
# PART 4: GOLD LAYER (FEATURES & RISK)
from pyspark.sql.functions import col, when, current_date, lit, max as spark_max

print("Step 3: Creating Gold Layer")

# 1. Read from Silver Delta
df_silver = spark.table("paysim_silver")

# A: Feature Engineering
# We are adding 3 critical features for fraud detection:
# 1. errorBalance: Detects mathematical anomalies in account balances.
# 2. hourOfDay: Fraud often happens at specific hours (extracted from step).
# 3. amountRatio: Fraudsters tend to empty accounts (amount / oldBalance).

df_gold = df_silver.withColumn("errorBalanceOrig", col("newbalanceOrig") + col("amount") - col("oldbalanceOrg")) \
                   .withColumn("errorBalanceDest", col("oldbalanceDest") + col("amount") - col("newbalanceDest")) \
                   .withColumn("type_index", when(col("type") == "TRANSFER", 0).otherwise(1)) \
                   .withColumn("hourOfDay", col("step") % 24) \
                   .withColumn("amountRatio", col("amount") / (col("oldbalanceOrg") + 0.001))

# Select Final Columns for ML
final_columns = [
    "step", 
    "type_index", 
    "amount", 
    "amountRatio",      
    "hourOfDay",      
    "oldbalanceOrg", 
    "newbalanceOrig", 
    "errorBalanceOrig",
    "oldbalanceDest", 
    "newbalanceDest", 
    "errorBalanceDest",
    "isFraud"
]

df_gold_ml = df_gold.select(final_columns)

print("Creating Gold Table: 'paysim_gold'")
spark.sql("DROP TABLE IF EXISTS paysim_gold")
df_gold_ml.write.format("delta").mode("overwrite").saveAsTable("paysim_gold")

# B: Risk Profile (Customer Behavioral Snapshot)
# Identify high-risk customers based on max transaction history.
# Strategy: Daily Full Refresh (Snapshot) to update the latest status for ALL customers.

df_risk_profile = df_silver.groupBy("nameOrig") \
    .agg(spark_max("amount").alias("max_txn_amount")) \
    .withColumn("risk_level", when(col("max_txn_amount") > 1000000, "High").otherwise("Low")) \
    .withColumn("effective_date", current_date()) \
    .withColumn("is_current", lit(True))

# Write to Delta Lake (Overwrite mode = Always keep the latest state)
# Note: We save ALL customers (Low & High) to maintain a complete Dimension Table, not just a blacklist.
df_risk_profile.write.format("delta").mode("overwrite").saveAsTable("dim_customer_risk")

# For demonstration, display only HIGH RISK customers
print("Displaying Sample of HIGH RISK Customers:")
display(df_risk_profile.filter(col("risk_level") == "High"))

print("Gold Layer Created Successfully")

In [0]:
# PART 5: MACHINE LEARNING PIPELINE & EVALUATION
from pyspark.sql.functions import col
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

print("Starting Optimized ML Pipeline")

# 1. Load Data from Delta
try:
    df_gold = spark.table("paysim_gold")
    gold_count = df_gold.count() 
    print(f"Loaded Gold Data. Total Rows: {gold_count:,}")
except Exception as e:
    print(f"Error: {e}")

# STEP A: STRICT TIME-SERIES SPLIT
# แยกอดีต (Train) ออกจากอนาคต (Test) ก่อนทำ Sampling
split_point = df_gold.approxQuantile("step", [0.8], 0.01)[0]
train_raw = df_gold.filter(col("step") <= split_point)
test_raw = df_gold.filter(col("step") > split_point)

# STEP B: STRATEGIC SAMPLING
# Train: เก็บ Fraud ครบ + สุ่ม Normal เหลือ 5%
train_data = train_raw.filter(col("isFraud") == 1).union(
    train_raw.filter(col("isFraud") == 0).sample(False, 0.05, seed=1234)
)

# Test: เก็บ Fraud ครบ + สุ่ม Normal เหลือ 5%
test_data = test_raw.filter(col("isFraud") == 1).union(
    test_raw.filter(col("isFraud") == 0).sample(False, 0.05, seed=5678)
)

print(f"Training Set: {train_data.count():,} rows")
print(f"Test Set: {test_data.count():,} rows")

# ใช้ Pipeline รวมขั้นตอนการทำ Feature และ Model เข้าด้วยกัน
feature_cols = [
    "type_index", "amount", "amountRatio", "hourOfDay",
    "oldbalanceOrg", "newbalanceOrig", "errorBalanceOrig",
    "oldbalanceDest", "newbalanceDest", "errorBalanceDest"
]

assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

rf = RandomForestClassifier(
    labelCol="isFraud", 
    featuresCol="features", 
    numTrees=10,
    maxDepth=7, 
    seed=1234
)

pipeline = Pipeline(stages=[assembler, rf])

# TRAINING
print("Training Random Forest via Pipeline")
pipeline_model = pipeline.fit(train_data)
print("Training Complete!")

# EVALUATION
predictions = pipeline_model.transform(test_data)

# คำนวณค่าเฉพาะ Class 1 (Fraud)
results = predictions.groupBy("isFraud", "prediction").count().collect()
tp = 0; fp = 0; fn = 0
for row in results:
    if row['isFraud'] == 1 and row['prediction'] == 1.0: tp = row['count']
    elif row['isFraud'] == 0 and row['prediction'] == 1.0: fp = row['count']
    elif row['isFraud'] == 1 and row['prediction'] == 0.0: fn = row['count']

# Metrics Summary
try:
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    evaluator = MulticlassClassificationEvaluator(labelCol="isFraud", predictionCol="prediction")
    accuracy = evaluator.evaluate(predictions, {evaluator.metricName: "accuracy"})
except:
    precision = recall = f1 = accuracy = 0

print(f"FINAL PERFORMANCE")
print("="*45)
print(f" Accuracy:  {accuracy:.4f}")
print(f" Precision: {precision:.2%}")
print(f" Recall:    {recall:.2%}")
print(f" F1-Score:  {f1:.4f}")
print("-" * 45)
print(f" Caught Fraud (TP): {tp}")
print(f" Missed Fraud (FN): {fn}")
print(f" False Alarm (FP):  {fp}")
print("="*45)

# แสดง Confusion Matrix
predictions.groupBy("isFraud", "prediction").count().show()

In [0]:
# FEATURE IMPORTANCE VISUALIZATION
importances = pipeline_model.stages[-1].featureImportances
feat_imp = pd.DataFrame(list(zip(feature_cols, importances.toArray())), 
                        columns=['Feature', 'Importance']).sort_values('Importance', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x="Importance", y="Feature", data=feat_imp, palette="magma")
plt.title('Why the Model thinks it is Fraud?', fontsize=14)
plt.show()